In [1]:
# First import some things we are gonna need
#import treecorr
import healpy as hp
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.table import Table
import numpy as np 
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import Angle
import pandas as pd
from angular_mcf import runComputation
from fit_cf import fitAngularCfMcf
import os 
import sys
sys.path.append('/home/krishna/Dropbox/updated_codes/')

from mypy_tree_angular import run_omega_computation
from mypy_tree_angular_mcf import run_omega_mcf_computation
from mypy_JKGenerator_singleRegn import make_jk_sample
from mypy_fit_omega import fit_omega_mcf

typp = 'all'
times_randoms = 1

In [2]:
# Open MAP 
# ==========================================
MAP = hp.read_map('/home/krishna/Documents/DES_MCF/dr2_hpix_4096_frac_detection_g.fits.fz')
conv = np.pi/180.
ra_min = -70.0*conv
ra_max = 110.*conv
dec_min = -70.*conv
dec_max = 10.*conv
# ========================
# ========================
rand_ra = np.random.uniform(ra_min, ra_max, 10**7)
rand_sindec = np.random.uniform(np.sin(dec_min), np.sin(dec_max), 10**7)
rand_dec = np.arcsin(rand_sindec)
# ========================

In [3]:
# Get the healpix indices of the randoms
HP_ind_rand = hp.ang2pix(4096, rand_ra/conv, rand_dec/conv, lonlat=True)

In [4]:
# Keep only those with randoms for which the value of the des mask 
# ===============================================
# ===============================================
# Keep local mask
# Get a mask - 1 inside the DES mask
DES_mask = np.zeros(len(MAP))
DES_mask[MAP>0] = 1
LOCAL_mask_rand = DES_mask[HP_ind_rand]

# Keep those with LOCAL_mask = 1
rand_ra_pos = rand_ra[LOCAL_mask_rand==1]
rand_dec_pos = rand_dec[LOCAL_mask_rand==1]

ra_angles = Angle(rand_ra_pos, unit=u.rad)
ra_angles_wrap=ra_angles.wrap_at(360.0* u.deg)
dec_angles = Angle(rand_dec_pos, unit=u.rad)

rand_ra_deg = ra_angles_wrap.degree
rand_dec_deg = dec_angles.degree

rand_ID = [i for i in range(len(rand_ra_deg))]
df_rand_full = pd.DataFrame({'ID': rand_ID, 'RA': rand_ra_deg, 'Dec': rand_dec_deg})

In [5]:
#realdata = Table.read('HSBG_sample_Thuruthipilly+24_v1.fits', format='fits').to_pandas()
#realdata = Table.read('DES_LSB+HSB_concatenatedTable.fits', format='fits').to_pandas()
#realdata = Table.read('DES_LSB+Mid_HSB_concatenatedTable.fits', format='fits').to_pandas()

#realdata.rename(columns = {'COADD_OBJECT_ID':'ID'}, inplace = True)
#realdata['COLOR_G_I']=realdata['MAG_AUTO_G']-realdata['MAG_AUTO_I']


In [6]:
working_dir = os.getcwd()


In [7]:

#LSB
realdata = Table.read('/home/krishna/Documents/DES_MCF/DES_LSB_Thuruthipilly+24_Sextractor.fits', format='fits').to_pandas()



realdata = realdata[realdata['MAG_AUTO_G'] < 19.85]

realdata['COLOR_G_I']=realdata['MAG_AUTO_G']-realdata['MAG_AUTO_I']
    
    
n_real = len(realdata)
n_rand = times_randoms*n_real

df_rand_full_sampled = df_rand_full.sample(150000)

cols_towrite_real = ['COADD_OBJECT_ID','RA','DEC','COLOR_G_I','MU_MEAN_MODEL_G','MAG_AUTO_G','MAG_AUTO_I']

df_subset = realdata[cols_towrite_real]

realdata_nandropped = df_subset.dropna()

realdata_towrite = realdata_nandropped

realdata_towrite.to_csv('real_galaxies', sep='\t',columns = cols_towrite_real, index=False, float_format='%f', header=[f'#{x}' if x == cols_towrite_real[0] else x for x in cols_towrite_real])


cols_towrite_random = ['ID','RA','Dec']
df_rand_full_sampled.to_csv('random_galaxies', sep='\t',columns = cols_towrite_random, index=False, float_format='%f',header=[f'#{x}' if x == cols_towrite_random[0] else x for x in cols_towrite_random])

    
#make_jk_sample(working_dir, njacks_ra=1, njacks_dec=30)
#run_omega_mcf_computation(working_dir)
#fit_omega_mcf(work_on_mcf = 1, fit_2pcf = 1, dir_name=working_dir)

In [8]:



realpops = ['COLOR_G_I','MU_MEAN_MODEL_G','MAG_AUTO_G','MAG_AUTO_I']

In [9]:
runComputation(realdata_towrite, realpops, df_rand_full_sampled, njacks_ra=1 , njacks_dec=30)

In [10]:
fitAngularCfMcf(realdata_towrite, realpops, sepmin= 0.01, sepmax=10.0, sepmin_tofit=0.4, sepmax_tofit=10.0, 
                to_svd_filter=0, to_hartlap_corr=0, fit_2pcf = 1, work_on_mcf = 1, 
                dir_name=os.getcwd())


SVD correction NOT done!

Hartlap correction NOT done!

--------------------------------------------

Fitting sample 2MPCF....

--------------------------------------------

Nr. of total bins:  22
Nr. of JK/BS copies:  30
Marks :  ['COLOR_G_I', 'MU_MEAN_MODEL_G', 'MAG_AUTO_G', 'MAG_AUTO_I']
Number of CF bins with non-nan values: 14
Number of bins used for CF fitting  7
Curve fitting parameters:
A = 0.19 +/- 0.03
gamma = 1.99 +/- 0.06

